Import Library

In [131]:
import pandas as pd
import numpy as np
from ast import literal_eval
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import time
import pickle as pk

In [101]:
movie=pd.read_csv("tmdb_5000_movies.csv")
credit=pd.read_csv("tmdb_5000_credits.csv")
t=time.time()

In [102]:
movies=movie.merge(credit)#,on="title")
# movies.head(2)

Colunms required for model

   #genres

   #Movie id
   
   #keywords

   #title

   #overview

   #cast
   
   #crew

In [103]:
movies=movies[['movie_id','title','overview','keywords',"genres",'cast','crew']]
# movies.head(2)

In [104]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
keywords    0
genres      0
cast        0
crew        0
dtype: int64

In [105]:
movies.dropna(inplace=True)

In [106]:
# movies.genres[0]

In [107]:
def Get_name(obj):
    L=[]
    for o in literal_eval(obj):
        L.append(o["name"].replace(" ",""))
    return L

def Get_name_cast(obj):
    L=[]
    count=0
    for o in literal_eval(obj):
        if count<=2:
            L.append(o["name"].replace(" ",""))
        else:
            break
        count+=1
    return L

def Get_name_director(obj):
    L=[]
    for o in literal_eval(obj):
        if 'Director' in o["job"]:
            L.append(o["name"].replace(" ",""))
            break
    return L

In [108]:
movies["genres"]=movies["genres"].apply(Get_name)
movies["keywords"]=movies["keywords"].apply(Get_name)

In [109]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [110]:
movies['cast']=movies['cast'].apply(Get_name_cast)

In [111]:
movies["crew"]=movies["crew"].apply(Get_name_director)

In [112]:
# movies.head()

In [113]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [114]:
movies=movies.drop(['overview','genres','keywords','cast','crew'],axis='columns')

In [115]:
# movies.head()

In [116]:
movies['tags']=movies['tags'].apply(lambda x:str(" ".join(x)).lower())

In [117]:
# movies.tags[0]

In [118]:
def Sterm(text):
    y=[]
    for i in text.split():
        y.append(PorterStemmer().stem(i))
    return " ".join(y)


movies['tags']=movies['tags'].apply(Sterm)

In [119]:
cv=CountVectorizer(max_features=5000,stop_words='english')
vactors=cv.fit_transform(movies['tags']).toarray()

In [162]:
similrity=cosine_similarity(vactors)

In [159]:
joblib.dump(similrity,"Model.pkl")
joblib.dump(movies,"Movies.pkl")
# joblib.dump()

['Movies.pkl']

In [158]:
joblib.dump(movies.title,"Movies_titles.txt")


['Movies_titles.txt']

In [153]:
movies.title[100:500]

100      The Curious Case of Benjamin Button
101                       X-Men: First Class
102    The Hunger Games: Mockingjay - Part 2
103                The Sorcerer's Apprentice
104                                 Poseidon
                       ...                  
495         Journey 2: The Mysterious Island
496      Cloudy with a Chance of Meatballs 2
497                               Red Dragon
498                                  Hidalgo
499                            Jack and Jill
Name: title, Length: 400, dtype: object

In [163]:
def Get_RMovie(text,size):
    M_Movies=[]
    similarity=joblib.load("Model.pkl")
    index=movies[movies['title']==text].index[0]
    distance=similarity[index]
    R_Movies=sorted(list(enumerate(distance)),key=lambda x:x[1],reverse=True)
    for r in R_Movies:
        title=movies['title'][r[0]]
        if len(M_Movies)<size and title!=text:
            M_Movies.append(title)
        elif len(M_Movies)>=size:
            break

    
    return M_Movies

In [164]:
# inp=input("Enter movie name: ")
size=5
movis=Get_RMovie("Hidalgo",size)
for i,m in enumerate(movis):
    print(f'{i+1} ) {m}')

# print(int(time.time())-int(t))

1 ) Cars 2
2 ) Puss in Boots
3 ) 127 Hours
4 ) Ulee's Gold
5 ) True Grit


In [170]:
index=list(movies['title'])



['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [173]:
with open("Movies Name.txt","w") as g:
    for i in index:
        try :
            g.write(i+"\n")

        except:
            continue